In [1]:
import json

with open('./task1.json', 'r') as f:
    test = json.load(f)

In [2]:
draft_ids = []
authors = []
coauthors = []
choices_2 = []
choices_3 = []
choices_4 = []
choices_5 = []

for instance in test:
    draft_ids.append(instance['folder_id'])
    authors.append(instance['author'])
    coauthors.append(instance['coauthor'])
    choices_2.append(instance['choices_2'])
    choices_3.append(instance['choices_3'])
    choices_4.append(instance['choices_4'])
    choices_5.append(instance['choices_5'])

In [13]:
import os

path = './task1'
draft_ids = ['2018']  # Modificamos esto para solo usar 2018
drafts = []

path = './task1'
for i in draft_ids:
    folder_path = os.path.join(path, str(i))
    if not os.path.exists(folder_path):
        print(f"El directorio {folder_path} no existe")
        continue
    files = os.listdir(folder_path)
    json_file = [file for file in files if file.endswith('EN.json')][0]
    with open(os.path.join(folder_path, json_file)) as f:
        draft = json.load(f)
    drafts.append(draft['Content'])

In [14]:
# If use Together API
from together import Together

your_model_name = 'meta-llama/Llama-3.3-70B-Instruct-Turbo-Free'
your_api_key = '574bf95f6b33074e60f284c2c6316e72c5ef8226ce8ce4650dacca4d51ce1050'
client = Together(api_key=your_api_key)

In [15]:
import random
from tqdm import tqdm

def run_task3_k_choices(client, drafts, authors, choices_k):
    results = []
    invalid_responses = []
    for i, (draft, author, choice_k) in tqdm(enumerate(zip(drafts, authors, choices_k))):
        system_prompt = f"""You are representing {author}, a country drafting a resolution for submission to the United Nations Security Council. 
        Your task is to review the draft resolution and select a coauthor from the following list: {', '.join(choice_k)}. 
        Respond only with the name of the chosen coauthor and provide no additional explanation."""
        user_prompt = f"""{author} is drafting a resolution to submit to the United Nations Security Council and is seeking a coauthor. 

        Review the following draft resolution and choose a coauthor from the list below. 
        Respond only with the name of the chosen country and provide no additional explanation.

        Available coauthors: {', '.join(choice_k)}

        Draft Resolution:
        {draft}

        Answer:
        """
        
        response = client.chat.completions.create(
            model=your_model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=20, 
            temperature=0.0
        )
        result = response.choices[0].message.content.strip()
        valid_response = choice_k
        if result not in valid_response:
            print(f'Invalid response: {result}')
            result = random.choice(valid_response)
            invalid_responses.append(i)
        results.append(result)
    return results, invalid_responses

In [16]:
results_2, invalid_responses_2 = run_task3_k_choices(client, drafts, authors, choices_2)

1it [00:03,  3.32s/it]


In [17]:
results_3, invalid_responses_3 = run_task3_k_choices(client, drafts, authors, choices_3)

1it [00:03,  3.99s/it]


In [18]:
results_4, invalid_responses_4 = run_task3_k_choices(client, drafts, authors, choices_4)

1it [00:07,  7.62s/it]


In [19]:
results_5, invalid_responses_5 = run_task3_k_choices(client, drafts, authors, choices_5)

1it [00:19, 19.67s/it]


In [20]:
def calculate_accuracy(results, ground_truth):
    correct = 0
    for i in range(len(results)):
        if results[i] == ground_truth[i]:
            correct += 1
    return correct/len(results)

In [21]:
accuracy_2 = calculate_accuracy(results_2, coauthors)
accuracy_3 = calculate_accuracy(results_3, coauthors)
accuracy_4 = calculate_accuracy(results_4, coauthors)
accuracy_5 = calculate_accuracy(results_5, coauthors)

In [22]:
print(f'Accuracy for 2 choices: {accuracy_2}')
print(f'Accuracy for 3 choices: {accuracy_3}')
print(f'Accuracy for 4 choices: {accuracy_4}')
print(f'Accuracy for 5 choices: {accuracy_5}')

Accuracy for 2 choices: 1.0
Accuracy for 3 choices: 0.0
Accuracy for 4 choices: 0.0
Accuracy for 5 choices: 0.0
